In [1]:
!pip install pytorch-lightning==1.9.2 transformers torchmetrics deepspeed nltk wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.2/826.2 KB 12.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.4/765.4 KB 42.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 KB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/Myashka/CQA_RLHF.git cqa

Cloning into 'cqa'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 417 (delta 29), reused 27 (delta 10), pack-reused 367
Receiving objects: 100% (417/417), 108.72 KiB | 4.35 MiB/s, done.
Resolving deltas: 100% (280/280), done.


In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install cloud-tpu-client==0.10 torch==1.13.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl tensorboard-plugin-profile
    !pip install google-api-python-client==1.12.1
    from google.colab import drive
    drive.mount('/content/drive')

In [3]:
import yaml

## Run train

In [ ]:
trainer_config = dict(
    seed=42,
    model_name="Myashka/125M_GPTneo_reward_base",
    wandb=dict(
        api="60fce56bfaec85b8d6bc78bfac2086891f5afe54",
        project_name="CQA_RLHF",
        args=dict(job_type="train", group="reward", name="125M-lr_1e_6-1_ws-api_usage-freezed_2"),
    ),
    data=dict(
        data_dir=r"/kaggle/input/python-qa-api-usage/1.0-data-div-ans-sep-api-usage.json",
        max_length=512,
        batch_size=8,
        pad_for_tpu = True,
        zero_question_labels=True,
    ),
    model_params=dict(
        learning_rate=1e-6,
        do_freeze=False,
        use_cache=False,
        warmup_steps_per_cent=0,
        adam_betas=[0.9, 0.95],
        weight_decay=0.001,
        do_compute_metrics = False,
        do_compute_bertscore = False,
        nr_frozen_epochs = 2,
    ),
    trainer=dict(
        checkpoint=dict(
            every_n_train_steps=1000,
            dirpath=r"/kaggle/working/Checkpoints",
            log_obg = 'val_loss',
            mode = 'min',

        ),
        ckpt_path=None,
        params=dict(
            accelerator="tpu",
            max_epochs=10,
            accumulate_grad_batches=1,
            gradient_clip_val=1,
            precision="16",
            val_check_interval=104,
            overfit_batches=0,  # 0 for train
            num_sanity_val_steps=2,
            log_every_n_steps = 20,
#             limit_train_batches=100,
#             strategy="deepspeed_stage_2_offload"
        ),
    ),
)

with open("trainer_config.yaml", "w") as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [ ]:
!python /kaggle/working/cqa/sft/train.py --config_file /kaggle/working/trainer_config.yaml

## Run test

In [ ]:
test_config = dict(
    seed=42,
    cuda=True,
    log_file = '/content/test-base-125M.csv',
    save_steps = 100,
    wandb=dict(
        api="60fce56bfaec85b8d6bc78bfac2086891f5afe54",
        args=dict(group="sft", job_type="test", name="125M-base-test"),
        project_name="CQA_RLHF",
        table_name="125M-base",
    ),
    model_name="EleutherAI/gpt-neo-125M",
    data=dict(data_dir=r"/content/drive/MyDrive/Colab Notebooks/vkr_data/data/1.0-data-div-ans-sep.json"),
    test_model_path=None,
    model_params=dict(use_cache=True, do_freeze=False),
    test_params=dict(do_compute_bertscore=False, do_compute_metrics=True),
    generate_params=dict(
        do_sample=True,
        top_k=50,
        top_p=0.9,
        # temperature=0,
        # num_return_sequences=0,
        no_repeat_ngram_size=2,
        max_new_tokens=512,
    ),
)

with open("test_config.yaml", "w") as outfile:
    yaml.dump(test_config, outfile, default_flow_style=False)

In [ ]:
!python /content/cqa/sft/test.py --config_file /content/test_config.yaml